# Bibliotecas

In [4]:
import random
import re
import nltk
import unidecode
import math
import time
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from numpy import mean
from nltk import tokenize
from string import punctuation
from string import ascii_letters
from nltk.stem import RSLPStemmer
from sklearn import metrics
from pandas_ods_reader import read_ods
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve
from sklearn.multiclass import OneVsRestClassifier
from sklearn.calibration import CalibratedClassifierCV
from pylab import plot, show, savefig, xlim, figure, ylim, legend, boxplot, setp, axes
from matplotlib import pyplot as plt

# Funções

In [232]:
#Função para ler os dados e realizar algumas definições
#Entrada: Nenhuma
#Saída: Dataframe com os dados e os tipos atribuídos

def leitura_e_configuracao():
    df = read_ods("base-de-dados-cientificos.ods",1)

    #Define os tipos de dados das colunas
    df['Palavras-chave'] = df['Palavras-chave'].astype(str)
    df['Título'] = df['Título'].astype(str)
    df['Resumo'] = df['Resumo'].astype(str)
    df['Categoria'] = df['Categoria'].astype(str)
    df['ID'] = df['ID'].astype(int)
    
    # atribuição dos ids a cada uma das categorias
    for abstract in df:
        df.loc[df['Categoria'] == 'Ciências Agrárias', 'ID_Categoria'] = 0
        df.loc[df['Categoria'] == 'Ciências Biológicas', 'ID_Categoria'] = 1
        df.loc[df['Categoria'] == 'Ciências Exatas e da Terra', 'ID_Categoria'] = 2
        df.loc[df['Categoria'] == 'Ciências Humanas', 'ID_Categoria'] = 3
        df.loc[df['Categoria'] == 'Ciências Sociais Aplicadas', 'ID_Categoria'] = 4
        df.loc[df['Categoria'] == 'Ciências da Saúde', 'ID_Categoria'] = 5
        df.loc[df['Categoria'] == 'Engenharias', 'ID_Categoria'] = 6
        df.loc[df['Categoria'] == 'Lingüística, Letras e Artes', 'ID_Categoria'] = 7
        df.loc[df['Categoria'] == 'Multidisciplinar', 'ID_Categoria'] = 8
    
    df['ID_Categoria'] = df['ID_Categoria'].astype(int) #tipo de dado da nova coluna criada
    df['Texto'] = df['Título'] + df['Resumo'] + df['Palavras-chave'] # Atributo texto é a união do título, resumo
                                                                     # e palavras-chave
    
    return df # retorna o dataframe
    
#Função para realizar o pré-processamento dos dados
#Entrada: Dataframe com os dados e o atributo a ser pré-processado
#Saída: Nenhuma

def pre_processamento(df, atributo):
    
    #letras minúsculas
    df[atributo] = [texto.lower() for texto in df[atributo]]

    #remoção de acentos
    df[atributo] = [unidecode.unidecode(texto) for texto in df[atributo]]

    #remoção de stop words
    stop_words = nltk.corpus.stopwords.words('portuguese')
    stop_words.extend(['artigo','sobre', 'resultado','objetivo','estudo','brasil', 'trabalho', 
                      'delineamento', 'experimental','analise'])
    lista_stopwords = []
    for abstract in df[atributo]:
        tokens = tokenize.WhitespaceTokenizer().tokenize(abstract)
        novo_texto = ' '.join([token for token in tokens if token not in stop_words])
        lista_stopwords.append(novo_texto)
    df[atributo] = lista_stopwords

    #remoção de pontuação
    pontuacoes = list(punctuation)
    pontuacao = []
    for texto in df[atributo]:
        tokens = tokenize.WhitespaceTokenizer().tokenize(texto)
        novo_texto = ' '.join([token for token in tokens if token not in pontuacoes])
        pontuacao.append(novo_texto)
    df[atributo] = pontuacao

    #Retira textos não desejados
    df[atributo] = df[atributo].apply(lambda x: re.sub(r'[0-9]','',x))
    df[atributo] = df[atributo].apply(lambda x: re.sub(r'[/(){}\[\]\#\|@,;.:-]',' ',x))
    df[atributo] = df[atributo].apply(lambda x: re.sub(r'\d+', '', x))

    #obtenção da raíz das palavras
    stemmer = RSLPStemmer()
    raizes = []
    for texto in df[atributo]:
        tokens = tokenize.WhitespaceTokenizer().tokenize(texto)
        novo_texto = ' '.join([stemmer.stem(token) for token in tokens])
        raizes.append(novo_texto)
    df[atributo] = raizes
    
#Função para realizar a extração de características
#Entrada: Dataframe e o atributo correspondente
#Saída vetor de características e os rótulos das categorias
    
def extracao_caracteristicas(df, atributo):
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range=(1, 2)) # aplicação do TF-IDF para
                                                                                        # extração de características
    vetor_caracteristicas = tfidf.fit_transform(df[atributo]).toarray() # obtenção do vetor de características
    rotulos = df.ID_Categoria
    return vetor_caracteristicas, rotulos

#Função para realizar a seleção de características
#Entrada: vetor de características e os rótulos das categorias
#Saída: vetor de características atualizado

def selecao_caracteristicas(vetor_caracteristicas, rotulos):
    chi2_test = SelectKBest(chi2,k=int((vetor_caracteristicas.shape[1]/2))) #Teste Qui-Quadrado usando para
                                                                            #selecionar metade das amostras
    vetor_caracteristicas = chi2_test.fit_transform(vetor_caracteristicas,rotulos) # obtenção do vetor de características
    return vetor_caracteristicas

#Função para realizar a validação do modelo usando 10-fold cross validation
#Entrada: Dataframe, classificador, vetor de características e os rótulos das categorias
#Saída: Dataframe com o resultado dos 10 folds

def validacao(df, modelo, vetor_caracteristicas, rotulos):
    
    vc_df = pd.DataFrame(index=range(10))
    entradas = []
    acuracias = cross_val_score(modelo, vetor_caracteristicas, rotulos, scoring='accuracy', cv=10)
    for num_fold, acuracia in enumerate(acuracias):
        entradas.append((num_fold, acuracia))
    vc_df = pd.DataFrame(entradas, columns=['num_fold', 'acuracia'])
    
    return vc_df

#Função para avaliar as dimensões de qualidade segundo os métodos de contaminação utilizando 10-fold cross validation
#e as condições experimentais de ruído
#Entrada: dimensão de qualidade a ser avaliada, atributo alvo, algoritmo de classificação selecionado
#Saída: lista contendo uma lista de 10 folds para cada condição experimental avaliada

def avaliacao_DQ(dimensao, atributo, classificador):
    ruidos = [0, 10, 20, 30, 40] # Grupo Controle, Grupo I, Grupo II, Grupo III, Grupo IV
    folds = []
    for ruido in ruidos:
        df = leitura_e_configuracao() # leitura/configuração do dataframe
        #contaminação
        if(dimensao == 'Acurácia-add'):
            contamina_acuracia(df, atributo, 'adicao', ruido)
        elif(dimensao == 'Acurácia-rm'):
            contamina_acuracia(df, atributo, 'remocao', ruido)
        elif(dimensao == 'Acurácia-subs'):
            contamina_acuracia(df, atributo, 'substituicao', ruido)
        elif(dimensao == 'Completude'):
            contamina_completude(df, atributo, ruido)
        elif(dimensao == 'Consistência'):
            contamina_consistencia(df, atributo, ruido)
        pre_processamento(df, atributo) # pré-processamento após a contaminação
        vetor_caracteristicas, rotulos = extracao_caracteristicas(df, atributo) #extração de características
        vetor_caracteristicas = selecao_caracteristicas(vetor_caracteristicas, rotulos)#seleção de características
        vc_df = validacao(df, classificador, vetor_caracteristicas, rotulos)#validação do modelo
        folds.append(vc_df.acuracia.values) # adiciona os 10 folds do grupo ruído à lista de folds
        
    return folds

#Função para gerar uma lista de valores em ASCII a partir de uma string
#Entrada: string desejada
#Saída: lista com os caracteres em ASCII

def gera_lista_valores(string):
    lista_valores = []
    lista_carac = []
    i = 0
    for i in range(len(string)):
        lista_carac.append(string[i])
    return [int(ord(carac)) for carac in lista_carac]

#Função para calcular a média de uma lista de valores
#Entrada: lista de valores
#Saída: média dos valores da lista

def calcula_media(lista_valores):
    soma = 0
    media = 0
    for valor in lista_valores:
        soma += valor
    return soma / len(lista_valores)

#Função para o cálculo do desvio padrão de uma lista de valores
#Entrada: média dos valores da lista e a lista de valores
#Saída: desvio padrão da lista de valores

def calcula_desvio(lista_valores, media):
    desvio = 0
    for valor in lista_valores:
        desvio += (valor - media)**2
    return (desvio / len(lista_valores))**(1/2)

#Função para obter a probabilidade de um caracter ser contaminado segundo uma distribuição de probabilidade
#Entrada: média e desvio padrão dos valores em ASCII correspondentes ao artigo e valor do caracter em ASCII sendo
#analisado
#Saída: probabilidade do caracter ser contaminado

def ruido_gaussiano(media, desvio, valor):
    print((1 / (desvio * math.sqrt(2 * math.pi))) * math.exp(-(((valor - media) ** 2) / (2 * (desvio ** 2)))))
    return (1 / (desvio * math.sqrt(2 * math.pi))) * math.exp(-(((valor - media) ** 2) / (2 * (desvio ** 2))))

#Função para realizar a contaminação da dimensão de qualidade Acurácia
#Entrada: dataframe, atributo escolhido, método(adição,remoção,substituição), porcentagem de ruído
#Saída: dataframe modificado

def contamina_acuracia(df, escolha_atributo, metodo, porc_ruido):
    lista_artigos = []
    
    if (porc_ruido == 0): # retorna caso não haja contaminação
        return
    
    qtd_artigos = int(df.shape[0] * (porc_ruido/100)) # quantidade de artigos que serão contaminados
    atributo_contaminado = escolha_atributo 
    i = 0
    
    while i < qtd_artigos: # laço para contaminar os artigos
        
        lista_carac = []
        lista_valores = []
        indice = random.choice(df.index) # artigo é selecionado de maneira aleatória
        
        if indice in lista_artigos: # verifica se o artigo já foi contaminado previamente
            while indice in lista_artigos:
                indice = random.choice(df.index)
                
        lista_valores = gera_lista_valores(df.loc[indice, atributo_contaminado]) # obtém os valores em ascii 
                                                                                 # em formato de lista
        media = calcula_media(lista_valores) # cálculo da média dos valores em ascii
        desvio = calcula_desvio(lista_valores, media) # desvio padrão dos valores em ascii
        
        for valor in lista_valores: # percorre os caracteres em ascii
            
            if ((ra) 
                and (chr(valor).isalpha())): # se o caracter for contaminado
                
                caracter = random.choice(ascii_letters) # obtém um caracter aleatório
                
                if(not caracter.isalpha()): # gera novo caracter enquanto este não for alfabético
                    while(not caracter.isalpha()):
                        caracter = random.choice(ascii_letters)
                                
                if(metodo == 'substituicao'): # método de substituição de caracteres
                    lista_carac.append(caracter) # adiciona o caracter aleatório no lugar do antigo na nova lista
                elif(metodo == 'adicao'): # método de adição de caracteres
                    lista_carac.extend([chr(valor), caracter]) # adiciona o caracter antigo junto com o aleatório
                                                               # à nova lista
                elif(metodo == 'remocao'): # remoção de caracteres
                    pass # caso o caracter seja contaminado, ele não é adicionado à lista de caracteres
            
            else: # caracter não será contaminado
                lista_carac.append(chr(valor)) # troca o valor em ascii pelo caracter correspondente
                                               # e o adiciona à lista
                
        df.loc[indice, atributo_contaminado] = ''.join([carac for carac in lista_carac]) # substitui o artigo pela
                                                                                         # lista resultante
        lista_artigos.append(indice) # adiciona o índice do artigo à lista de artigos contaminados
        i+=1

#Função para realizar a contaminação da dimensão de qualidade Completude
#Entrada: dataframe, atributo escolhido, porcentagem de ruído
#Saída: dataframe modificado        

def contamina_completude(df, escolha_atributo, porc_ruido):
    lista_artigos = []
    #lista_atributos = ['Título', 'Resumo', 'Palavras-chave']
    if(porc_ruido == 0):
        return
    qtd_artigos = int(df.shape[0] * (porc_ruido/100))
    i = 0
    atributo_contaminado = escolha_atributo
    while i < qtd_artigos:
        indice = random.choice(df.index) # sorteia um artigo aleatório
        if indice in lista_artigos: # se o artigo já foi contaminado
            while indice in lista_artigos: # procura por um novo artigo enquanto estiver na lista dos que já foram contaminados
                indice = random.choice(df.index)
        df[atributo_contaminado] = df[atributo_contaminado].replace([df[atributo_contaminado][indice]], '') # apaga o atributo
        lista_artigos.append(indice) # adiciona o artigo contaminado à lista de artigos que já foram contaminados
        i += 1

#Função para realizar a contaminação da dimensão de qualidade Consistência
#Entrada: dataframe, atributo escolhido, porcentagem de ruído
#Saída: dataframe modificado         
        
def contamina_consistencia(df, escolha_atributo1, porc_ruido):
    lista_artigos = []
    lista_atributos = ['Título', 'Resumo', 'Palavras-chave']
    if(porc_ruido == 0):
        return
    porc_ruido /= 2
    qtd_artigos = int(df.shape[0] * (porc_ruido/100)) # número de artigos a serem contaminados
    i = 0
    atributo_contaminado1 = escolha_atributo1
    atributo_contaminado2 = random.choice(lista_atributos)
    while i < qtd_artigos:
        indice1 = random.choice(df.index)
        indice2 = random.choice(df.index)
        if(df['ID_Categoria'][indice1] == df['ID_Categoria'][indice2]):
                while(df['ID_Categoria'][indice1] == df['ID_Categoria'][indice2]):
                        indice1 = random.choice(df.index)
        
        else:
            temp = df[atributo_contaminado1][indice1]
            if(atributo_contaminado2 == atributo_contaminado1):
                while(atributo_contaminado2 == atributo_contaminado1):
                    atributo_contaminado2 = random.choice(lista_atributos)
            df[atributo_contaminado1] = df[atributo_contaminado1].replace([df[atributo_contaminado1][indice1]], [df[atributo_contaminado2][indice2]])
            df[atributo_contaminado2] = df[atributo_contaminado2].replace([df[atributo_contaminado2][indice2]], temp)
            i+=1
            
#Função para obter a melhor divisão do conjunto de treinamento/teste e exibir graficamente
#Entrada: classificador
#Saída: indicação gráfica da melhor porcentagem de amostras usadas para o treinamento do modelo de classificação
            
def melhor_divisao(modelo):
    lista_acuracias = []
    lista_porc = []
    i = 0.01 # Começa em 1%
    while i < 1: # Enquanto não chegar a 100%
        X_train, X_test, y_train, y_test = train_test_split(vetor_caracteristicas, rotulos, train_size=i, random_state=0)
        modelo.fit(X_train, y_train) # faz o treinamento usando a porcentagem i como conjunto de treino
        y_pred = modelo.predict(X_test)
        lista_acuracias.append(accuracy_score(y_test, y_pred)*100)
        lista_porc.append(i*100)
        i += 0.01
    
    #configuração do gráfico a ser exibido
    fig, ax = plt.subplots()
    ax.plot(lista_porc, lista_acuracias, 'black')
    plt.xlabel('Conjunto de Treino (%)')
    plt.ylabel('Acurácia (%)')
    indice = lista_acuracias.index(max(lista_acuracias))
    plt.scatter(lista_porc[indice], max(lista_acuracias), c='red', linewidths=2)
    plt.axhline(y=max(lista_acuracias), xmax=lista_porc[indice]/105, linestyle='--', color="red")
    plt.axvline(x=lista_porc[indice], ymax=max(lista_acuracias)/(max(lista_acuracias)+5), linestyle='--', color="red")
    plt.text(lista_porc[indice]+2, max(lista_acuracias)+0.5, '({:.1f}, {:.1f})'.format(lista_porc[indice], max(lista_acuracias)))
    plt.margins(0.1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.show()
    
#Função para gerar as curvas ROC e disponibilizá-las em formato gráfico
#Entrada: classificador e melhor divisão do conjunto de treino
#Saída: representação gráfica das curvas ROC para o modelo escolhido
    
def gera_curvas_roc(modelo, cjto_treino):

    #treinamento do modelo utilizando o melhor split possível
    X_train, X_test, y_train, y_test = train_test_split(vetor_caracteristicas, rotulos, train_size=(cjto_treino/100), random_state=0)
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    pred_prob = modelo.predict_proba(X_test) #obtenção das probabilidades que serão utilizadas para construir as
                                             #curvas ROC
    
    FPR = {} #dicionário contendo os pares chave-valor dos falsos positivos (1-Especificidade)
    TPR = {} #dicionárito contendo os pares chave-valor dos verdadeiros positivos (Sensibilidade)
    limite = {}

    num_classes = 9

    for i in range(num_classes):
        FPR[i], TPR[i], limite[i] = roc_curve(y_test, pred_prob[:, i], pos_label=i) #obtenção dsa curvas ROC
                                                                                    #para as nove classes
    
    # plotando a área sob a curva e configurando exbição do gráfico
    fig, ax = plt.subplots()
    plt.plot(FPR[0], TPR[0], color='#e41a1c', linestyle='-', label='CA vs Resto', linewidth=3)
    plt.plot(FPR[1], TPR[1], color='#377eb8', linestyle='-', label='CB vs Resto',linewidth=3)
    plt.plot(FPR[2], TPR[2], color='#4daf4a', linestyle='-', label='CET vs Resto', linewidth=3)
    plt.plot(FPR[3], TPR[3], color='#984ea3', linestyle='-', label='CH vs Resto', linewidth=3)
    plt.plot(FPR[4], TPR[4], color='#ff7f00', linestyle='-', label='CSA vs Resto', linewidth=3)
    plt.plot(FPR[5], TPR[5], color='#ffff33', linestyle='-', label='CS vs Resto', linewidth=3)
    plt.plot(FPR[6], TPR[6], color='#a65628', linestyle='-', label='E vs Resto', linewidth=3)
    plt.plot(FPR[7], TPR[7], color='#f781bf', linestyle='-', label='LLA vs Resto', linewidth=3)
    plt.plot(FPR[8], TPR[8], color='#999999', linestyle='-', label='M vs Resto', linewidth=3)
    plt.xlabel('Taxa de Falsos Positivos (1 - Especificidade)', weight='book')
    plt.ylabel('Taxa de Verdadeiros Positivos (Sensibilidade)', weight='book')
    ax.xaxis.set_tick_params(width=2)
    ax.yaxis.set_tick_params(width=2)
    leg = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., fontsize=26, shadow=True,
                     fancybox=True)
    leg.get_frame().set_linewidth(3)
    for i in range(num_classes):
        leg.get_lines()[i].set_linewidth(5)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.setp(ax.spines.values(), linewidth=2)
    fig.set_size_inches(20, 10)
    plt.show()
    
#Função para criar o box-plot contendo os dados dos 10 folds obtidos por meio do 10-fold cross validation para cada
#um dos algoritmos
#Entrada: valores do eixo x, valores do eixo y e fonte dos dados
#Saída: Representação gráfica do box-plot
    
def cria_box_plot(eixo_x, eixo_y):

    #configuração do gráfico
    fig, ax = plt.subplots()
    flierprops = dict(marker='*', markerfacecolor='black', markersize=8,
                  markeredgecolor='none')
    medianprops = dict(linestyle='-', linewidth=1.1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    #modelos a serem validados
    modelos = [
    LinearSVC(random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=0, solver='liblinear')
]
    
    vc_df = pd.DataFrame(index=range(10 * len(modelos))) #dataframe para os folds da validação cruzada

    entradas = []
    
    #10-fold cross validation
    for modelo in modelos:
        nome_modelo = modelo.__class__.__name__
        if(nome_modelo == "LinearSVC"):
            nome_modelo = "MVS"
        elif(nome_modelo == "MultinomialNB"):
            nome_modelo = "NB"
        elif(nome_modelo == "LogisticRegression"):
            nome_modelo = "RL"
        acuracias = cross_val_score(modelo, vetor_caracteristicas, rotulos, scoring='accuracy', cv=10)
        for num_fold, acuracia in enumerate(acuracias):
            entradas.append((nome_modelo, num_fold, acuracia))
    
    #Adequação do dataframe
    vc_df = pd.DataFrame(entradas, columns=['Classificador', 'Num_Fold', 'Acurácia (%)'])
    vc_df['Acurácia (%)'] = [acuracia * 100 for acuracia in vc_df['Acurácia (%)']]

    #Construção do boxplot
    sns.boxplot(x=eixo_x, y=eixo_y, palette='bright', data=vc_df, orient='v', width=0.3,
           saturation=1, linewidth=1, showcaps=False, flierprops=flierprops, medianprops=medianprops)
    plt.show()
    
    return vc_df
    
    
#Função para realizar a leitura e configuração dos resultados dos folds
    
def leitura_e_configuracao_resultados():

    df_resultados = read_ods("resultadosFoldsFormatado.ods",1)

    #Define os tipos de dados das colunas
    df_resultados['Classificador'] = df_resultados['Classificador'].astype(str)
    df_resultados['Método'] = df_resultados['Método'].astype(str)
    df_resultados['Atributo'] = df_resultados['Atributo'].astype(str)
    df_resultados['Fold'] = df_resultados['Fold'].astype(int)
    df_resultados['Grupo'] = df_resultados['Grupo'].astype(str)
    df_resultados['Acurácia'] = df_resultados['Acurácia'].astype(float)
    
    return df_resultados
    
#Função para definir cores do Box-plot    

def setCorBoxPlot(bp, cor):
    for item in ['whiskers', 'fliers', 'medians', 'caps']:
        plt.setp(bp[item], color='black')
    plt.setp(bp["boxes"], facecolor=cor)

#Função para criar os box-plots para os algoritmos sob as condições experimentais de ruído e um método de contaminação
#Entrada: dataframe, atributo contaminado, método de contaminação, eixo do gráfico construído
#Saída: gráfico com quatro quadrantes, onde cada um contém os box-plots dos classificadores
#e representam o atributo contaminado segundo as condições experimentais de ruído

def cria_box_plots(df, atributo, metodo, ax):
    
    #configurações do box-plot
    flierprops = dict(marker='*', markerfacecolor='black', markersize=10,
                  markeredgecolor='none')
    medianprops = dict(linestyle='-', linewidth=1.1)
    
    classificadores = ['MVS','NBM','RL'] #classificadores
    posicoes = [[1,2,3,4,5],[8,9,10,11,12],[15,16,17,18,19]] #posições dos box-plots
    grupos_ruido = ['C','I','II','III','IV'] #condições experimentais
    
    for i in range(len(classificadores)): # para cada classificador
        
        lista_ruidos = []
        for j in range(len(grupos_ruido)):
            #filtra o dataframe com as colunas desejadas
            df_ruido = pd.DataFrame(df[(df['Grupo'] == grupos_ruido[j]) 
                                                          & (df['Classificador'] == classificadores[i])
                                                          & (df['Atributo'] == atributo) 
                                                          & (df['Método'] == metodo)])
            lista_ruidos.append(df_ruido['Acurácia']) #inclui apenas os valores de acurácia na lista
        
        bp = ax.boxplot(lista_ruidos, positions=posicoes[i], patch_artist=True, flierprops=flierprops,
                    medianprops=medianprops, showcaps=False, labels=grupos_ruido) #constroi o box-plot do classificador
        #define as cores
        if i == 0:
            setCorBoxPlot(bp,'blue')
        elif i == 1:
            setCorBoxPlot(bp,'orange')
        elif i == 2:
            setCorBoxPlot(bp,'green')

    #configurações dos eixos
    ax.set_yticks([20,30,40,50,60,70])
    ax.set_xlim(0,20)
    ax.set_ylim(19,72)
    
    #adição do rótulo correspondente ao classificador utilizado no eixo x
    secax = ax.secondary_xaxis(-0.08)
    secax.xlim = (0,20)
    secax.spines['bottom'].set_visible(False)
    secax.set_xticks([3,10,17])
    secax.set_xticklabels(classificadores)
    secax.tick_params(axis='x', length=0)
    
    #adição do titulo ao subplot indicando o atributo correspondente
    ax.set_title('{}'.format(atributo), y=0.9, pad=-10, loc='right', position=(0.97, 0.5))
    
    #configurações do texto do gráfico
    font = {'family':'sans-serif',
            'weight':'book',
            'size': 16}
    
    plt.rc('font', **font)

#Função para criar o grid e os 4 quadrantes onde serão colocados os box-plots
#Saída: os axis representando cada um dos quadrantes

def set_grid(metodo):
    fig = plt.figure(figsize=(20,10)) #criação da figura
    plt.ylabel("Acurácia (%)", labelpad=40) #título eixo y
    #omissão dos ticks dos eixos para não sobressair
    plt.yticks([])
    plt.xticks([])
    gs = fig.add_gridspec(2, 2, hspace=0, wspace=0) #especificações do grid
    (ax1, ax2), (ax3, ax4) = gs.subplots(sharex='col', sharey='row') #compartilhamento das linhas e colunas
    
    return ax1,ax2,ax3,ax4


# Testes-Contaminação

In [3]:
%%time
atributo = 'Título'
metodos = ['Acurácia-add', 'Completude', 'Consistência']
modelo = LogisticRegression(random_state=0, solver='liblinear')

for metodo in metodos:
    lista_folds = []
    i=0
    arquivo = open("resultados/rl/acc/{}-{}-ACC.txt".format(atributo, metodo),"w")
    arquivo_folds = open("resultados/rl/folds/{}-{}-FOLDS.txt".format(atributo, metodo),"w")
    for i in range(10):
        folds = avaliacao_DQ(metodo, atributo, modelo)
        lista_folds.append(folds)
    lista_media_folds = [sum(val)/10 for val in zip(*lista_folds)]
    for media_fold in lista_media_folds:
        arquivo.write("{}\n".format((sum(media_fold)/len(media_fold))*100))
        for fold in media_fold:
            arquivo_folds.write("{}\n".format(fold*100))
    print("{}: {} ✔\n".format(atributo, metodo))
    arquivo.close()
    arquivo_folds.close()

Título: Acurácia-add ✔

Título: Completude ✔

Título: Consistência ✔

CPU times: user 6min 48s, sys: 1min 58s, total: 8min 47s
Wall time: 5min 9s


# Pré-processamento/Extração/Seleção

In [ ]:
df = leitura_e_configuracao()

In [ ]:
pre_processamento(df, 'Texto')

In [ ]:
vetor_caracteristicas, rotulos = extracao_caracteristicas(df, 'Texto')

In [ ]:
vetor_caracteristicas = selecao_caracteristicas(vetor_caracteristicas, rotulos)

## Testes

In [233]:
df_teste = leitura_e_configuracao()

In [234]:
artigo = df_teste.sample(n=1)

In [235]:
contamina_acuracia(artigo, 'Palavras-chave', 'adicao', 100)

0.016744287105261643
0.01682800195398392
0.013710048811204073
0.01293148111242553
0.016318837603451555
0.015649199602325213
0.016318837603451555
0.010393384920965096
0.016861511792332712
0.015082069267329344
0.01293148111242553
0.01682800195398392
0.00031036533874857905
0.015082069267329344
0.016318837603451555
0.01293148111242553
0.013710048811204073
0.01476642588893438
0.016631093655174265
0.01682800195398392
0.015082069267329344
0.016861511792332712
0.016881789647332248
0.01476642588893438
0.001148716376369452
0.00031036533874857905
0.015649199602325213
0.01682800195398392
0.016631093655174265
0.012525150780582953
0.015376819306708726
0.016318837603451555
0.015082069267329344
0.01476642588893438
0.01332703738235297
0.016861511792332712
0.001148716376369452
0.00031036533874857905
0.01332703738235297
0.016318837603451555
0.015376819306708726
0.016898589761187372
0.016318837603451555
0.01476642588893438
0.01332703738235297
0.01682800195398392
0.001148716376369452
0.00031036533874857905

In [227]:
artigo['Palavras-chave'].values

array(['ahmFiJnkoOáAcYitdDoVsw; dKeasmseeDdDexnLtFazçMãTod; fKiBtJaLsHel; mgiynQekrgaeidsb oirIgpâIneipcloMsX'],
      dtype=object)

# Validação

## Validação cruzada

In [ ]:
vc_df = cria_box_plot('Classificador', 'Acurácia (%)')

In [ ]:
vc_df

In [ ]:
vc_df['Acurácia'] = vc_df['Acurácia (%)']
acuracias = vc_df.groupby('Classificador').Acurácia.mean()
acuracias.sort_values(ascending=False)

In [ ]:
acuracias_desvios = vc_df.groupby('Classificador').Acurácia.std()
acuracias_desvios.sort_values(ascending=False)

In [ ]:
%%time
modelo = MultinomialNB()
acuracias = cross_val_score(modelo, vetor_caracteristicas, rotulos, scoring='accuracy', cv=10)
print(mean(acuracias)*100)

## Leave-one-out

In [ ]:
%%time
loo = LeaveOneOut()
modelo = LogisticRegression(random_state=0, solver='liblinear')
acuracia = cross_val_score(modelo, vetor_caracteristicas, rotulos, scoring='accuracy', cv=loo)
print(mean(acuracia)*100)

## Divisão treino/teste

In [ ]:
melhor_divisao(LinearSVC(random_state=0))

In [ ]:
%%time
X_train, X_test, y_train, y_test = train_test_split(vetor_caracteristicas, rotulos, train_size=0.63, random_state=0)
modelo = LinearSVC()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)

## Matriz de confusão

In [ ]:
modelo = SVC(kernel="linear", probability=True)


X_train, X_test, y_train, y_test = train_test_split(vetor_caracteristicas, rotulos, train_size=0.63, random_state=0)
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

#criação de dicionário (categoria,id)
category_id_df = df[['Categoria', 'ID_Categoria']].drop_duplicates().sort_values('ID_Categoria')
category_to_id = dict(category_id_df.values)
#criação de dicionário (id, categoria)
id_to_category = dict(category_id_df[['ID_Categoria', 'Categoria']].values)

#criação das siglas
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Agrárias', 'Categoria'] = 'CA'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Biológicas', 'Categoria'] = 'CB'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Exatas e da Terra', 'Categoria'] = 'CET'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Humanas', 'Categoria'] = 'CH'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências Sociais Aplicadas', 'Categoria'] = 'CSA'
category_id_df.loc[category_id_df['Categoria'] == 'Ciências da Saúde', 'Categoria'] = 'CS'
category_id_df.loc[category_id_df['Categoria'] == 'Engenharias', 'Categoria'] = 'E'
category_id_df.loc[category_id_df['Categoria'] == 'Lingüística, Letras e Artes', 'Categoria'] = 'LLA'
category_id_df.loc[category_id_df['Categoria'] == 'Multidisciplinar', 'Categoria'] = 'M'

#matriz de confusão
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat.diagonal()/conf_mat.sum(axis=0)
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(conf_mat, annot=True, fmt='d',
xticklabels=category_id_df.Categoria.values, yticklabels=category_id_df.Categoria.values)
plt.ylabel('Real')
plt.xlabel('Previsto')
plt.show()

In [ ]:
#imprime as métricas
print(metrics.classification_report(y_test, y_pred, target_names=df['Categoria'].unique()))

## Curvas-ROC

In [ ]:
from sklearn.svm import SVC

modelo_calibrado = CalibratedClassifierCV(LinearSVC(random_state=0), cv=10, ensemble=False)

gera_curvas_roc(modelo_calibrado, 63)
gera_curvas_roc(MultinomialNB(), 82)
gera_curvas_roc(LogisticRegression(random_state=0, solver='liblinear'), 82)

## Box-Plots

In [ ]:
df_resultados = leitura_e_configuracao_resultados()

In [ ]:
ax1,ax2,ax3,ax4 = set_grid('Acurácia')

cria_box_plots(df_resultados, 'Título', 'ACC', ax1)
cria_box_plots(df_resultados, 'Resumo', 'ACC', ax2)
cria_box_plots(df_resultados, 'Palavras-chave', 'ACC', ax3)
cria_box_plots(df_resultados, 'Texto', 'ACC', ax4)

In [ ]:
ax1,ax2,ax3,ax4 = set_grid('Completude')

cria_box_plots(df_resultados, 'Título', 'COMP', ax1)
cria_box_plots(df_resultados, 'Resumo', 'COMP', ax2)
cria_box_plots(df_resultados, 'Palavras-chave', 'COMP', ax3)
cria_box_plots(df_resultados, 'Texto', 'COMP', ax4)

In [ ]:
ax1,ax2,ax3,ax4 = set_grid('Consistência')

cria_box_plots(df_resultados, 'Título', 'CONS', ax1)
cria_box_plots(df_resultados, 'Resumo', 'CONS', ax2)
cria_box_plots(df_resultados, 'Palavras-chave', 'CONS', ax3)
cria_box_plots(df_resultados, 'Texto', 'CONS', ax4)

## Two-Way ANOVA

In [ ]:
import pingouin as pg

In [ ]:
df_resultados = leitura_e_configuracao_resultados()
df_resultados = df_resultados.drop(columns='unnamed.2')

In [ ]:
def two_way_anova(df, metodo, atributo):
    
    df_anova = pd.DataFrame(df[(df['Atributo'] == atributo) & (df['Método'] == metodo)])
    
    aov = pg.anova(dv='Acurácia', between=['Classificador', 'Grupo'], data=df_anova,
             detailed=True)
    
    return df_anova, aov

In [ ]:
def media_anova(df):
    
    grupos = ['C','I','II','III','IV']
    lista_media_grupos = []
    
    for grupo in grupos:
        df_grupo = pd.DataFrame(df[(df['Grupo'] == grupo)])
        lista_media_grupos.append(df_grupo['Acurácia'].mean())
    
    return lista_media_grupos

In [ ]:
atributos = ['Título', 'Resumo','Palavras-chave','Texto']
metodos = ['ACC', 'COMP', 'CONS']

for metodo in metodos:
    print('\n{}\n'.format(metodo))
    for atributo in atributos:
        
        df_anova, anova = two_way_anova(df_resultados,metodo,atributo)
        lista_medias = media_anova(df_anova)
        
        total = np.array([['Total',sum(anova['SS'].values), sum(anova['DF'].values)]])
        total_anova = pd.DataFrame(data=total, columns=['Source','SS', 'DF'])
        anova = anova.append(total_anova)
        
        display('{}'.format(atributo), anova)
        print("Médias: {}".format(lista_medias))
        
